In [ ]:
'''
Train data 350 ,Test data 150 
epochs 25
Used EfficientNet series models
'''

In [1]:
#content google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import tensorflow as tf
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import SGD
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
!pip install tensorflow-addons
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score
import glob
#select tensorflow's version
%tensorflow_version 2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 35.3 MB/s 
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
#形状設定
IMAGE_SHAPE=(224,224)

#リアルタイムにデータ拡張しながら，テンソル画像データのバッチを生成する。(1/255 メモリ消費の削減)
image_generator=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.3
    )

In [4]:
# HDF5フォーマットでモデルを保存するために必要
!pip install -q pyyaml h5py  

In [5]:
#各データセットのディレクトリを選択し、全画像をリサイズする。
#ディレクトリへのバスを受け取り、拡張・正規化したデータバッチを生成する。
batch_size=32

train_generator = image_generator.flow_from_directory(
    '/content/drive/MyDrive/data1/train',
    target_size=(224, 224),
    #class_mode='binary',
    batch_size=batch_size,
    subset='training',
)

val_generator = image_generator.flow_from_directory(
    '/content/drive/MyDrive/data1/train',
    target_size=(224, 224),
    #class_mode='binary',
    batch_size=batch_size,
    subset='validation'
)

Found 1400 images belonging to 4 classes.
Found 600 images belonging to 4 classes.


In [6]:
#各データセットのディレクトリを選択し、全画像をリサイズする。
#ディレクトリへのバスを受け取り、拡張・正規化したデータバッチを生成する。
#train_data=image_generator.flow_from_directory('/content/drive/MyDrive/data/train',target_size=IMAGE_SHAPE)
test_data=image_generator.flow_from_directory('/content/drive/MyDrive/data1/test',target_size=IMAGE_SHAPE)
gizi_data=image_generator.flow_from_directory('/content/drive/MyDrive/data1/train',target_size=IMAGE_SHAPE)
val_data=image_generator.flow_from_directory('/content/drive/MyDrive/data1/val',target_size=IMAGE_SHAPE)

Found 100 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 48 images belonging to 4 classes.


In [7]:
#各データセットの画像バッチ形状と、ラベルのバッチ形状を表示する
#擬似データはラベル付けない
for image_train_batch,label_train_batch in train_generator:
  print("Image batch shape:",image_train_batch.shape)
  print("Label batch shape:",label_train_batch.shape)
  break

for image_train_batch,label_train_batch in train_generator:
  print("Image batch shape:",image_train_batch.shape)
  print("Label batch shape:",label_train_batch.shape)
  break

for image_val_batch,label_val_batch in val_data:
  print("Image batch shape:",image_val_batch.shape)
  print("Label batch shape:",label_val_batch.shape)
  break

for image_test_batch,label_test_batch in test_data:
  print("Image batch shape:",image_test_batch.shape)
  print("Label batch shape:",label_test_batch.shape)
  break

Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)
Image batch shape: (32, 224, 224, 3)
Label batch shape: (32, 4)


In [8]:
#モデル設定
MODEL_DIR='/content/drive/MyDrive/result11'
if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)

checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )
#filepath:モデルファイルを保存する場所を選択
#monitor:監視するメトリック名
#mode:ファイルを上書きするかどうか
#save_weights_only:モデルのウェイトのみを保存する
#verbose:1、冗長モード                                             

In [9]:
checkpoint1=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model01-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [10]:
checkpoint2=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model02-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [11]:
checkpoint3=tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_DIR,"model03-{epoch:02d}.h5"),
                                              monitor='val_loss',
                                              mode='min',
                                              save_weights_only=True,
                                              verbose=1
                                              )

In [12]:
#訓練プロセスの可視化
#カスタムコールバックを作成して、すべてのエポックでのトレーニングの進行状況を視覚化します
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses=[]
    self.batch_acc=[]
    
  def on_train_batch_end(self,batch,logs=1e-12):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

In [13]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetv2
feature_extractor_url="https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer=hub.KerasLayer(feature_extractor_url,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer.trainable=False

#上位の分類層を接合する
ebc1=Sequential([feature_extractor_layer,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense (Dense)               (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [14]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetb1
feature_extractor_url_b1="https://tfhub.dev/google/efficientnet/b1/feature-vector/1"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer_b1=hub.KerasLayer(feature_extractor_url_b1,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer_b1.trainable=False

#上位の分類層を接合する
ebc2=Sequential([feature_extractor_layer_b1,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              6575232   
                                                                 
 dense_1 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 6,580,356
Trainable params: 5,124
Non-trainable params: 6,575,232
_________________________________________________________________


In [15]:
#使用するモデルの選択（特徴抽出器/分類器）_ efficientnetb2
feature_extractor_url_b2="https://tfhub.dev/google/efficientnet/b2/feature-vector/1"

#入力画像サイズを決定する(モデルに適したサイズを設定)
feature_extractor_layer_b2=hub.KerasLayer(feature_extractor_url_b2,input_shape=(512,512,3))

#学習済み重みを固定する
feature_extractor_layer_b2.trainable=False

#上位の分類層を接合する
ebc3=Sequential([feature_extractor_layer_b2,layers.Dense(train_generator.num_classes,activation='softmax')])

#モデルの概要を確認する
ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1408)              7768562   
                                                                 
 dense_2 (Dense)             (None, 4)                 5636      
                                                                 
Total params: 7,774,198
Trainable params: 5,636
Non-trainable params: 7,768,562
_________________________________________________________________


In [16]:
batch_stats_callback=CollectBatchStats()
batch_size=32
epochz=25
epoch=10
#steps_per_epoch
s_p_e=np.ceil(train_generator.samples/batch_size)

model1

In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/25
44/44 [==============================] - 1248s 27s/step - loss: 0.3338 - acc: 0.9062
Epoch 2/25
44/44 [==============================] - 1191s 27s/step - loss: 0.1357 - acc: 0.9688
Epoch 3/25
44/44 [==============================] - 1199s 27s/step - loss: 0.2017 - acc: 0.9375
Epoch 4/25
44/44 [==============================] - 1215s 28s/step - loss: 0.1053 - acc: 1.0000
Epoch 5/25
44/44 [==============================] - 1219s 28s/step - loss: 0.0734 - acc: 1.0000
Epoch 6/25
44/44 [==============================] - 1217s 28s/step - loss: 0.1701 - acc: 0.9375
Epoch 7/25
44/44 [==============================] - 1216s 28s/step - loss: 0.1037 - acc: 0.9688
Epoch 8/25
44/44 [==============================] - 1201s 27s/step - loss: 0.1646 - acc: 0.9688
Epoch 9/25
44/44 [==============================] - 1192s 27s/step - loss: 0.0446 - acc: 1.0000
Epoch 10/25
44/44 [==============================] - 1192s 27s/step - loss: 0.1317 - acc: 0.9688
Epoch 11/25
44/44 [====================

教師モデル学習

In [18]:
#上位分類層を接合
ebc1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_3 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [ ]:
ebc1.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[checkpoint1,batch_stats_callback],
         )

Epoch 1/25
44/44 [==============================] - ETA: 0s - loss: 0.5296 - acc: 0.8125 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
44/44 [==============================] - 1766s 40s/step - loss: 0.5296 - acc: 0.8125 - val_loss: 0.7923 - val_acc: 0.6633
Epoch 2/25
44/44 [==============================] - ETA: 0s - loss: 0.1519 - acc: 0.9688 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
44/44 [==============================] - 1780s 41s/step - loss: 0.1519 - acc: 0.9688 - val_loss: 0.6317 - val_acc: 0.7617
Epoch 3/25
44/44 [==============================] - ETA: 0s - loss: 0.2626 - acc: 0.9062 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
44/44 [==============================] - 1742s 40s/step - loss: 0.2626 - acc: 0.9062 - val_loss: 0.5495 - val_acc: 0.7933
Epoch 4/25
44/44 [==============================] - ETA: 0s - loss: 0.0692 - acc: 1.0000 
Epoch 4: saving model to /content/drive/MyDrive/result11/mode

In [19]:
#上位分類層を接合
ebc1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc1.summary()

ebc1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_4 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [19]:
#擬似ラベル作成
history=ebc1.fit(train_generator,
                  epochs=epochz,
                  steps_per_epoch=s_p_e,
                  validation_data=gizi_data,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

Epoch 1/25
44/44 [==============================] - ETA: 0s - loss: 0.4640 - acc: 0.8438 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
44/44 [==============================] - 1882s 43s/step - loss: 0.4640 - acc: 0.8438 - val_loss: 0.5271 - val_acc: 0.8080
Epoch 2/25
44/44 [==============================] - ETA: 0s - loss: 0.2164 - acc: 0.9688 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
44/44 [==============================] - 1938s 45s/step - loss: 0.2164 - acc: 0.9688 - val_loss: 0.3651 - val_acc: 0.8850
Epoch 3/25
44/44 [==============================] - ETA: 0s - loss: 0.3255 - acc: 0.9688 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
44/44 [==============================] - 1982s 46s/step - loss: 0.3255 - acc: 0.9688 - val_loss: 0.3278 - val_acc: 0.8915
Epoch 4/25
44/44 [==============================] - ETA: 0s - loss: 0.5127 - acc: 0.8125 
Epoch 4: saving model to /content/drive/MyDrive/result11/mode

In [20]:
#疑似ラベルの予測
gizi_label1=ebc1.predict(gizi_data)

63/63 [==============================] - 1161s 18s/step


In [20]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model1=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model1=Sequential([stochastic_Depth])

#モデルノイズ（dropout）の設定と適用
model1=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                           
                  ])

In [21]:
#model1
model1.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model1.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_5 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [21]:
#model1
history=model1.fit(gizi_data,
                  epochs=epochz,
                  steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                  validation_data=val_generator,
                  callbacks=[checkpoint1,batch_stats_callback]
                  )

Epoch 1/25
63/63 [==============================] - ETA: 0s - loss: 0.3183 - acc: 0.9062 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
63/63 [==============================] - 2462s 38s/step - loss: 0.3183 - acc: 0.9062 - val_loss: 0.3795 - val_acc: 0.8667
Epoch 2/25
63/63 [==============================] - ETA: 0s - loss: 0.3995 - acc: 0.8438 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
63/63 [==============================] - 2437s 39s/step - loss: 0.3995 - acc: 0.8438 - val_loss: 0.2570 - val_acc: 0.9217
Epoch 3/25
63/63 [==============================] - ETA: 0s - loss: 0.0805 - acc: 1.0000 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
63/63 [==============================] - 2826s 45s/step - loss: 0.0805 - acc: 1.0000 - val_loss: 0.2142 - val_acc: 0.9417
Epoch 4/25
63/63 [==============================] - ETA: 0s - loss: 0.1481 - acc: 0.9375 
Epoch 4: saving model to /content/drive/MyDrive/result11/mode

1

In [22]:
history=model1.fit(train_generator,
                   epochs=15,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint1,batch_stats_callback]
                   )

#y_prob = model1.predict(val_data)
#y_pred1 = y_prob.argmax(axis=-1)
#y_test1=np.argmax(y_test, axis=1)
# accuracy
#print(model1+" accuracy: ",accuracy_score(y_test1,y_pred1))
#del model1 , y_pred1, y_test1

Epoch 1/15
44/44 [==============================] - ETA: 0s - loss: 0.5578 - acc: 0.6875 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
44/44 [==============================] - 1839s 41s/step - loss: 0.5578 - acc: 0.6875 - val_loss: 0.7695 - val_acc: 0.7100
Epoch 2/15
44/44 [==============================] - ETA: 0s - loss: 0.3567 - acc: 0.9062 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
44/44 [==============================] - 1772s 41s/step - loss: 0.3567 - acc: 0.9062 - val_loss: 0.5893 - val_acc: 0.7683
Epoch 3/15
44/44 [==============================] - ETA: 0s - loss: 0.2613 - acc: 0.8438 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
44/44 [==============================] - 1812s 41s/step - loss: 0.2613 - acc: 0.8438 - val_loss: 0.5929 - val_acc: 0.7633
Epoch 4/15
44/44 [==============================] - ETA: 0s - loss: 0.1271 - acc: 0.9688 
Epoch 4: saving model to /content/drive/MyDrive/result11/mode

In [23]:
history=model1.fit(train_generator,
                   epochs=10,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint1,batch_stats_callback]
                   )

Epoch 1/10
44/44 [==============================] - ETA: 0s - loss: 0.0937 - acc: 0.9688 
Epoch 1: saving model to /content/drive/MyDrive/result11/model01-01.h5
44/44 [==============================] - 1885s 43s/step - loss: 0.0937 - acc: 0.9688 - val_loss: 0.4897 - val_acc: 0.8217
Epoch 2/10
44/44 [==============================] - ETA: 0s - loss: 0.0599 - acc: 0.9688 
Epoch 2: saving model to /content/drive/MyDrive/result11/model01-02.h5
44/44 [==============================] - 1866s 43s/step - loss: 0.0599 - acc: 0.9688 - val_loss: 0.4528 - val_acc: 0.8150
Epoch 3/10
44/44 [==============================] - ETA: 0s - loss: 0.0382 - acc: 1.0000 
Epoch 3: saving model to /content/drive/MyDrive/result11/model01-03.h5
44/44 [==============================] - 1879s 43s/step - loss: 0.0382 - acc: 1.0000 - val_loss: 0.4311 - val_acc: 0.8267
Epoch 4/10
44/44 [==============================] - ETA: 0s - loss: 0.0578 - acc: 0.9688 
Epoch 4: saving model to /content/drive/MyDrive/result11/mode

In [24]:
# HDF5フォーマットでモデルを保存するために必要
!pip install -q pyyaml h5py  

In [25]:
model1.save('/content/drive/MyDrive/result11/model1.h5/')

model2

In [26]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

Epoch 1/25
44/44 [==============================] - 151s 3s/step - loss: 0.6115 - acc: 0.8125
Epoch 2/25
44/44 [==============================] - 132s 3s/step - loss: 0.4387 - acc: 0.8750
Epoch 3/25
44/44 [==============================] - 127s 3s/step - loss: 0.2415 - acc: 0.9688
Epoch 4/25
44/44 [==============================] - 133s 3s/step - loss: 0.2311 - acc: 0.9688
Epoch 5/25
44/44 [==============================] - 130s 3s/step - loss: 0.2239 - acc: 0.9688
Epoch 6/25
44/44 [==============================] - 131s 3s/step - loss: 0.4294 - acc: 0.9375
Epoch 7/25
44/44 [==============================] - 132s 3s/step - loss: 0.2926 - acc: 0.8438
Epoch 8/25
44/44 [==============================] - 130s 3s/step - loss: 0.2514 - acc: 0.8750
Epoch 9/25
44/44 [==============================] - 137s 3s/step - loss: 0.1813 - acc: 0.9688
Epoch 10/25
44/44 [==============================] - 133s 3s/step - loss: 0.1829 - acc: 0.9375
Epoch 11/25
44/44 [==============================] - 128s 3

教師モデル学習

In [17]:
#上位分類層を接合
ebc2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_3 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [18]:
ebc2.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint2],
         )

Epoch 1/25
44/44 [==============================] - ETA: 0s - loss: 0.5132 - acc: 0.7500 
Epoch 1: saving model to /content/drive/MyDrive/result11/model02-01.h5
44/44 [==============================] - 1213s 27s/step - loss: 0.5132 - acc: 0.7500 - val_loss: 0.7484 - val_acc: 0.7150
Epoch 2/25
44/44 [==============================] - ETA: 0s - loss: 0.1623 - acc: 0.9688 
Epoch 2: saving model to /content/drive/MyDrive/result11/model02-02.h5
44/44 [==============================] - 1195s 27s/step - loss: 0.1623 - acc: 0.9688 - val_loss: 0.6366 - val_acc: 0.7867
Epoch 3/25
44/44 [==============================] - ETA: 0s - loss: 0.1560 - acc: 0.9688 
Epoch 3: saving model to /content/drive/MyDrive/result11/model02-03.h5
44/44 [==============================] - 1221s 28s/step - loss: 0.1560 - acc: 0.9688 - val_loss: 0.5256 - val_acc: 0.7983
Epoch 4/25
44/44 [==============================] - ETA: 0s - loss: 0.3024 - acc: 0.9062 
Epoch 4: saving model to /content/drive/MyDrive/result11/mode

In [19]:
#上位分類層を接合
ebc2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc2.summary()

ebc2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dense_4 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 207,620,956
Trainable params: 5,124
Non-trainable params: 207,615,832
_________________________________________________________________


In [20]:
#擬似ラベル作成
history=ebc2.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[checkpoint2,batch_stats_callback]
                 )

Epoch 1/25
44/44 [==============================] - ETA: 0s - loss: 0.4573 - acc: 0.8438 
Epoch 1: saving model to /content/drive/MyDrive/result11/model02-01.h5
44/44 [==============================] - 2006s 46s/step - loss: 0.4573 - acc: 0.8438 - val_loss: 0.5594 - val_acc: 0.8110
Epoch 2/25
44/44 [==============================] - ETA: 0s - loss: 0.2231 - acc: 1.0000 
Epoch 2: saving model to /content/drive/MyDrive/result11/model02-02.h5
44/44 [==============================] - 2009s 46s/step - loss: 0.2231 - acc: 1.0000 - val_loss: 0.3710 - val_acc: 0.8880
Epoch 3/25
44/44 [==============================] - ETA: 0s - loss: 0.4272 - acc: 0.8750 
Epoch 3: saving model to /content/drive/MyDrive/result11/model02-03.h5
44/44 [==============================] - 2057s 47s/step - loss: 0.4272 - acc: 0.8750 - val_loss: 0.3076 - val_acc: 0.9025
Epoch 4/25
44/44 [==============================] - ETA: 0s - loss: 0.1158 - acc: 0.9688 
Epoch 4: saving model to /content/drive/MyDrive/result11/mode

KeyboardInterrupt: ignored

In [ ]:
gizi_label2=ebc2.predict(gizi_data)

In [ ]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model2=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model2=Sequential([stochastic_Depth])



model2=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                  ])



In [ ]:
#model2
model2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model2.summary()


In [ ]:
#model2
history=model2.fit(gizi_data,
                   epochs=15,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )


In [ ]:
history=model2.fit(gizi_data,
                   epochs=10,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

2

In [ ]:
history=model2.fit(train_generator,
                   epochs=15,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
history=model2.fit(train_generator,
                   epochs=10,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[checkpoint2,batch_stats_callback]
                   )

In [ ]:
model2.save('/content/drive/MyDrive/result11/model2.h5/')

model3

In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         #validation_data=val_generator,
         callbacks=[batch_stats_callback],
         )

In [ ]:
#上位分類層を接合
ebc3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])



教師モデル作成

In [ ]:
ebc3.fit(train_generator,
         epochs=epochz,
         steps_per_epoch=s_p_e,
         validation_data=val_generator,
         callbacks=[batch_stats_callback,checkpoint3],
         )

In [ ]:
#上位分類層を接合
ebc3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax')])

ebc3.summary()

ebc3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
#擬似ラベル作成
history=ebc3.fit(train_generator,
                 epochs=epochz,
                 steps_per_epoch=s_p_e,
                 validation_data=gizi_data,
                 callbacks=[batch_stats_callback,checkpoint3],
                 )

In [ ]:
gizi_label3=ebc3.predict(gizi_data)

In [ ]:
feature_extractor_layer.trainable=False
#入力画像ノイズ(rand augmentation)の設定
data_augmentation=Sequential([layers.RandomRotation(2.10)])
#モデルノイズ（stochastic depth）の設定
stochastic_Depth=Sequential([tfa.layers.StochasticDepth(0.35)])

#モデルにrand augmentationを適用
model3=Sequential([data_augmentation])
#モデルにstochastic depthを適用
model3=Sequential([stochastic_Depth])

model3=Sequential([feature_extractor_layer,
                           layers.Dense(train_generator.num_classes,activation='softmax'),
                           #layers.Dropout(0.2)
                  ])

In [ ]:
#model3
model3.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['acc'])
model3.summary()

In [ ]:
#model3
history=model3.fit(gizi_data,
                   epochs=15,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint3,batch_stats_callback]
                   )

In [ ]:
history=model3.fit(gizi_data,
                   epochs=10,
                   steps_per_epoch=np.ceil(gizi_data.samples/batch_size),
                   validation_data=val_generator,
                   callbacks=[checkpoint3,batch_stats_callback]
                   )

3

In [ ]:
history=model3.fit(train_generator,
                   epochs=15,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

In [ ]:
history=model3.fit(train_generator,
                   epochs=10,
                   steps_per_epoch=s_p_e,
                   validation_data=val_generator,
                   callbacks=[batch_stats_callback,checkpoint3]
                   )

In [ ]:
#モデル保存
model3.save('/content/drive/MyDrive/result11/model3.h5/')

In [ ]:
from google.colab import runtime
runtime.unassign()

Accuracy for each model

In [ ]:
#content google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model1 = keras.models.load_model('/content/drive/MyDrive/result11/model1.h5/')
model2 = keras.models.load_model('/content/drive/MyDrive/result11/model2.h5/')
model3 = keras.models.load_model('/content/drive/MyDrive/result11/model3.h5/')

In [ ]:
train_datagenerator = ImageDataGenerator(rescale = 1.0/255,
                                        shear_range = 0.2,
                                        zoom_range = 0.5,
                                        horizontal_flip = True,
                                        rotation_range=10,
                                        width_shift_range=0.2,
                                        brightness_range=[0.2,1.2]
                                        )
test_datagenerator = ImageDataGenerator(rescale = 1.0/255)

training_set = train_datagenerator.flow_from_directory('/content/drive/MyDrive/data1/train/',
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

class_dict = training_set.class_indices
print(class_dict)

li = list(class_dict.keys())
print(li)

valid_set = test_datagenerator.flow_from_directory('/content/drive/MyDrive/data1/test',
                                                  target_size = (512,512),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

# valid_set
x_test, y_test = valid_set.next() 

In [ ]:
# predicting an image
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np

class votingClassifer:
    'votingClassifer class'


    def __init__(self, estimators,mode="hard",weight=None,show_info="percent"):
        
        if mode not in ["hard","soft"]:
            raise Exception("the mode should be 'hard' or 'soft'")
            
        self.estimators=estimators
        self.mode=mode
        self.weight=weight
        self.show_info=show_info  
            
    def predict(self,x_test):        
        if self.mode=="hard":
            return self.votingClassifer_hard(self.estimators, x_test, show_info=self.show_info)
        else:
            return self.votingClassifer_soft(self.estimators, x_test, weight=self.weight, show_info=self.show_info)

    def getNumberDiff(self, index_classes, n):  
        
        for x in index_classes:
            if index_class_prefer != x:
                return x    
    
    def getNumberElse(self, index_classes, n):

        indices = [i for i, value in enumerate(index_classes) if value != n]
        counts = np.bincount(indices)
        ind=np.argmax(counts)
        n1=(indices == ind).sum()
 
                
        return n1
    
    # hard voting
    def votingClassifer_hard(self,estimators,x_test,show_info='percent'):
        
        if show_info not in ["info","percent","nothing"]:
            raise Exception("the attribut 'show_info' should be 'info' or 'percent','nothing'")            
            
        
        cpt=0
        index_classes_glob, class_names_glob, probs_glob=[],[],[]
        N=len(x_test)
        for x in x_test:
            index_classes, class_names, probs=[],[],[]
            for model in estimators:
                img = np.expand_dims(x, axis=0)
                # make a prediction
                y_prob = model.predict(img)[0]
                probabilty = y_prob.flatten()
                max_prob = probabilty.max()
                y_classes = y_prob.argmax(axis=-1)
                index_class, class_name, prob = y_classes,li[y_classes],max_prob
                index_classes.append(index_class)
                class_names.append(class_name)
                probs.append(prob)
            index_classes, class_names, probs = np.array(index_classes), np.array(class_names), np.array(probs)    
            counts = np.bincount(index_classes)
            index_class_prefer=np.argmax(counts)
 
            n1=(index_classes == index_class_prefer).sum()
    
            if n1 == 1:
                print("\n Each estimator predict a different class")
                prob = probs.max()
                indice = [i for i, value in enumerate(probs) if value == prob][0]
                class_name = class_names[indice]

                
            elif n1 == len(estimators)/2 and len(estimators)/2 == self.getNumberElse(index_classes, index_class_prefer):
                
                print("\n the half-estimators predict a class and the other estimators predict a different class")
                
                indices1 = [i for i, value in enumerate(index_classes) if value == index_class_prefer]
                sum2=0
                for ind in indices1:
                    sum2+=probs[ind]
                
                prob1=sum2/len(indices1)
                
                n2=self.getNumberDiff(index_classes, index_class_prefer)      
                
                indices2 = [i for i, value in enumerate(index_classes) if value == n2]
                sum2=0
                for ind in indices2:
                    sum2+=probs[ind]
                
                prob2=sum2/len(indices2)
                
                if prob1 < prob2:
                    prob=prob2
                    indice = [i for i, value in enumerate(index_classes) if value == n2][0]
                    class_name = class_names[indice]
                    
                else:
                    prob=prob1
                    indice = [i for i, value in enumerate(index_classes) if value == index_class_prefer][0]
                    class_name = class_names[indice]
                
            else:
                
                sum1=0
                nbr=0
                for i, index in zip(range(len(probs)),index_classes):
                    if index_class_prefer== index:
                        sum1+=probs[i]
                        nbr+=1
                        
                prob=sum1/nbr
                indice = index_class_prefer
                class_name = li[index_class_prefer]

            
            if show_info=="info":
                cpt+=1
                print("\rpercent: {:.2f}%, li[{}]:{} --> {}".format(cpt*100/N,index_class_prefer, li[index_class_prefer], prob), end='')   
            elif show_info=="percent":
                cpt+=1
                print("\rpercent: {:.2f}%".format(cpt*100/N), end='')
          
                
            index_classes_glob.append(indice)
            class_names_glob.append(class_name)
            probs_glob.append(prob)

        return np.array(index_classes_glob), np.array(class_names_glob), np.array(probs_glob) 
    
   # soft voting 
    def votingClassifer_soft(self, estimators, x_test, weight=None, show_info="percent"):
        
        if show_info not in ["info","percent","nothing"]:
            raise Exception("the attribut 'show_info' should be 'info' or 'percent','nothing'")  

        if weight is None :
            weight=np.ones(len(estimators))

        if len(weight) != len(estimators):
            raise Exception("number of models and wheight should be equals")    

        cpt=0 
        # get number of classes 
        x = image.img_to_array(x_test[0])
        x = np.expand_dims(x, axis=0)
        y_prob = estimators[0].predict(x) 
        num_classes =  y_prob.shape[1]

        Tab=np.zeros(num_classes)# num_classes
        index_classes_glob, class_names_glob, probs_glob=[],[],[]
        N=len(x_test)
        for x in x_test:
            index_classes, class_names, probs= [], [], []
            for model in estimators:
                img = np.expand_dims(x, axis=0)
                # make a prediction
                y_prob = model.predict(img)[0]
                idxs = np.argsort(y_prob)
                # loop over the indexes of the high confidence class labels
                for (index, value) in enumerate(idxs):
                    # build the label and draw the label on the image
                    #label = "{}) {}[{}]: {:.2f}%".format(index,li[value],value, y_prob[value] * 100)
                    #print(label)
                    Tab[value]=y_prob[value]

                probs.append(Tab)

            probs=np.array(probs) # probs.shape: (3, 38)

            proba=[]   

            div=sum(weight)
            for i in range(len(probs[0])):
                s=0
                for j in range(len(weight)):
                    s+= probs[j][i] * weight[j] 
                s=s/div
                proba.append(s)

            proba=np.array(proba)
            max_prob=max(proba)

            indices=[i for i, value in enumerate(proba) if value == max_prob]

            index_class, class_name, prob=indices[0],li[indices[0]],max_prob 

            index_classes_glob.append(index_class)
            class_names_glob.append(class_name)
            probs_glob.append(prob) 

            if show_info=="info":
                cpt+=1
                print("percent: {:.2f}%, li[{}]:{} --> {}%".format(cpt*100/N,index_class, li[index_class], prob))   
            elif show_info=="percent":
                cpt+=1
                print("\rpercent: {:.2f}%".format(cpt*100/N), end='')



        return np.array(index_classes_glob), np.array(class_names_glob), np.array(probs_glob)  

In [ ]:
from sklearn.base import is_classifier, is_regressor
models_names=['model1','model2','model3']
print("model name\t estimator name\t is_regressor\t is_classifier")    
for estimator , model_name in zip([model1,model2,model3],models_names):
        print("{}\t   {}     \t {}      \t {}".format(model_name,estimator.__class__.__name__,
                                              is_regressor(estimator),
                                              is_classifier(estimator)
                                              ))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import numpy as np
models=[model1,model2,model3]
model_names=["model1","model2",'model3']
for model,model_name in zip(models,model_names):
            
    y_prob = model.predict(x_test)
    y_pred1 = y_prob.argmax(axis=-1)
    y_test1=np.argmax(y_test, axis=1)
    # accuracy
    print(model_name+" accuracy: ",accuracy_score(y_test1,y_pred1))
    del model , y_pred1, y_test1
    #Precision
    #print('Precision:', precision_score(y_test,y_pred1))
    #Recall
    #print('Recall:', recall_score(y_test1,y_pred1))
    #F-measure
    #print("Classification report")
    #print(classification_report(y_test1, y_pred1))

In [ ]:
# valid_set
x_test, y_test = valid_set.next() 

#VotingClassifier(hard)
estimators = [model1,model2,model3]
vc=votingClassifer(estimators=estimators,mode="hard",show_info="percent")
index_classes, class_names, probs=vc.predict(x_test)

accuracy

In [ ]:
# inverse_to_categorical inverser format binary to format indexation
# datagenerator use the methode to_categorical for labelsation to frmat binary 
# the method inverce of to_categorical is argmax
import numpy as np
y_test1=np.argmax(y_test, axis=1)
y_pred1=index_classes

# accuracy
from sklearn.metrics import accuracy_score
print("votingClassifer(hard) accuracy : ",accuracy_score(y_test1,y_pred1))